# Una solución empresarial completa

## Ahora llevaremos nuestro proyecto del día 1 al siguiente nivel

### DESAFÍO EMPRESARIAL:

Crear un producto que genere un folleto para una empresa que se utilizará para posibles clientes, inversores y posibles reclutas.

Se nos proporcionará un nombre de empresa y su sitio web principal.

Consulte el final de este cuaderno para ver ejemplos de aplicaciones empresariales del mundo real.

Y recuerde: ¡siempre estoy disponible si tiene problemas o ideas! No dude en comunicarse conmigo.

In [22]:
# imports
# Si esto falla, verifica que esté ejecutándose desde un entorno "activado" con (llms) en el símbolo del sistema

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [23]:
# Inicialización y constantes and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print("La clave de API parece buena")
else:
    print("¿Puede haber un problema con tu clave API? ¡Visita el cuaderno de resolución de problemas!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

La clave de API parece buena


In [24]:
# La clase para representar una Página Web

class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado, ahora con enlaces
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "Sin título"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Título de la Web:\n{self.title}\nContenido de la Web:\n{self.text}\n\n"

In [25]:
link_system_prompt = "Se te proporciona una lista de enlaces que se encuentran en una página web. \
Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto de Marketing sobre la solución Siigo Nube, \
como enlaces a quien va dirigido, beneficios, los planes.\n"
link_system_prompt += "Debes responder en JSON como en este ejemplo:"
link_system_prompt += """
{
    "links": [
        {"type": "Pagina De la solucion", "url": "https://url.completa/aqui/va/sistema-administrativo-en-la-nube"},
        {"type": "Pagina de verticales": "url": "https://otra.url.completa/software-administrativo-nube-consultorio-dental"}
    ]
}
"""

In [26]:
def get_links_user_prompt(website):
    user_prompt = f"Aquí hay una lista de enlaces de la página web {website.url} - "
    user_prompt += "Por favor, decide cuáles de estos son enlaces web relevantes para un folleto de Marketing sobre la solución Siigo Nube. Responde con la URL https completa en formato JSON. \
No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico, otros países, Distribuidores, ni paginas duplicadas.\n"
    user_prompt += "Sólo incluye los links del dominio aspel.com.mx\n"
    user_prompt += "Links (puede que algunos sean links relativos):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [27]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

## Segundo paso: ¡crea el folleto!

Reúne todos los detalles en otro mensaje para GPT4-o

In [28]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Links encontrados:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [29]:
system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa\
y crea un folleto breve de Marketing de alto impacto para obtener nuevos clientes sobre la solución Siigo Nube.\
Incluye detalles sobre los beneficios, a quien va dirigido, los precios de los planes."


In [30]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"Estás mirando una empresa llamada: {company_name}\n"
    user_prompt += f"Aquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de Marketing de la solución Siigo Nube en Markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncar si tiene más de 20.000 caracteres
    return user_prompt

In [31]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [32]:
create_brochure("Aspel", "https://www.aspel.com.mx/sistema-administrativo-en-la-nube")

Links encontrados: {'links': [{'type': 'Página de la solución', 'url': 'https://www.aspel.com.mx/sistema-administrativo-en-la-nube'}, {'type': 'Beneficios del sistema administrativo', 'url': 'https://www.aspel.com.mx/sae?pag=beneficios'}, {'type': 'Beneficios del administrativo COI', 'url': 'https://www.aspel.com.mx/coi?pag=beneficios'}, {'type': 'Beneficios del administrativo NOI', 'url': 'https://www.aspel.com.mx/noi?pag=beneficios'}, {'type': 'Beneficios de Facturación en la Nube', 'url': 'https://www.aspel.com.mx/facturacion-nube'}, {'type': 'Lista de precios', 'url': 'https://www.aspel.com.mx/lista-precios'}, {'type': 'Prueba gratis SAE', 'url': 'https://www.aspel.com.mx/prueba-gratis-sae'}, {'type': 'Prueba gratis COI', 'url': 'https://www.aspel.com.mx/prueba-gratis-coi'}, {'type': 'Prueba gratis NOI', 'url': 'https://www.aspel.com.mx/prueba-gratis-noi'}, {'type': 'Prueba gratis Caja', 'url': 'https://www.aspel.com.mx/prueba-gratis-caja'}, {'type': 'Prueba gratis Facture', 'url':

```markdown
# 🌟 ¡Descubre Siigo Nube de Aspel! 🌟

### El ERP en la Nube clave para el futuro de tu negocio

**Siigo Nube Gestión** es un sistema administrativo que transforma la manera de operar pequeñas y medianas empresas (pymes), ofreciendo una solución integral para la administración y contabilidad en la Nube.

---

## 🚀 **Beneficios de Siigo Nube:**

- **Visión Completa del Negocio:** Accede a un panorama de ventas, compras, inventarios y cobranza en tiempo real para tomar decisiones estratégicas.
  
- **Colaboración Efectiva:** Facilita el trabajo conjunto con contadores, centralizando la información en una plataforma segura.
  
- **Relación con Clientes:** Identifica a tus clientes clave y sus necesidades, mejorando la lealtad comercial.
  
- **Cumplimiento Normativo:** Mantente al día con las regulaciones del SAT, gracias a actualizaciones automáticas del sistema.
  
- **Fácil de Usar:** Interfaz intuitiva que no requiere ser un experto para navegar y obtener información clave.

---

## 🎯 **¿A quién va dirigido?**

- **Emprendedores y PYMES** que buscan controlar su negocio de manera eficiente.
  
- **Contadores y Despachos Contables** que desean optimizar el trabajo colaborativo con sus clientes.

- **Sectores específicos** como consultorios dentales, servicios de belleza y más.

---

## 💰 **Planes y Precios:**

1. **Siigo Nube Gestión Inicial**
   - **Precio:** $367 / mes o $3,199.20 / año (20% de descuento) 
   - **Características:**
     - 2 usuarios
     - 1 Almacén
     - Timbrado ilimitado
     - Reportes y más.

2. **Siigo Nube Gestión Avanzado**
   - **Precio:** $510 / mes o $4,447.20 / año (20% de descuento)
   - **Características:**
     - Todo en el plan inicial más integración más mejoras en los reportes.

3. **Siigo Nube Gestión Premium**
   - **Precio:** $878 / mes o $7,663.20 / año (20% de descuento)
   - **Características:**
     - 3 usuarios
     - Almacenes ilimitados
     - Todas las funciones avanzadas.

*Todos los precios son sin IVA.*

---

## 📞 **Contacta a nuestras Ventas:**
- 📞 **Teléfono:** (+52) 834 358 001
- 🕐 **Horario de Atención:** Lunes a viernes de 9am a 6pm

---

## 🚀 **¡Prueba gratis y conoce más!**
Visita nuestro sitio o contacta a un asesor para obtener una demostración gratuita y explore cómo Siigo Nube puede llevar tu gestión administrativa al siguiente nivel.

---

### **Aspel - Simplificando tu gestión empresarial en la Nube.**
```